In [ ]:
# ============================================================================
# LangChain RAG Tool Setup - Cell 1: Imports and LLM Configuration
# ============================================================================

# ============================================================================
# Core Imports
# ============================================================================
import os
from dotenv import load_dotenv

# ============================================================================
# Text Processing & Embeddings
# ============================================================================
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# ============================================================================
# Vector Stores & Retrieval
# ============================================================================
from langchain_community.vectorstores import FAISS
from langchain_core.tools import create_retriever_tool

# ============================================================================
# Language Models (LLMs)
# ============================================================================
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# ============================================================================
# Prompts & Agents
# ============================================================================
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.agents import create_agent

# ============================================================================
# Checkpoint & Memory (for agent state)
# ============================================================================
from langgraph.checkpoint.memory import InMemorySaver

# Load environment variables from .env file
load_dotenv()

# ============================================================================
# API Key Configuration
# ============================================================================
# Retrieve API keys for Gemini and OpenAI from environment variables
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Validate that both API keys are present
assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."

# Display partial keys to confirm successful loading (for security, only show first 6 chars)
print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")

# ============================================================================
# Initialize Language Models (LLMs)
# ============================================================================
# Initialize Google's Gemini LLM with temperature=0 for deterministic responses
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,  # Deterministic output (no randomness)
    model="gemini-2.5-flash",  # Using the faster Gemini model
    google_api_key=GEMINI_API_KEY
)

# Initialize OpenAI's GPT-3.5-turbo LLM
openai_llm = ChatOpenAI(
    temperature=0,  # Deterministic output
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# ============================================================================
# Create a Simple Prompt Template
# ============================================================================
# Define a reusable prompt template for company name generation
prompt = PromptTemplate(
    input_variables=["product"],  # Variable to be filled dynamically
    template="Give me a creative name for a company that makes {product}?",
)

# ============================================================================
# Example 1: LLM Chain (Prompt → LLM)
# ============================================================================
# Create a chain: prompt template piped to OpenAI LLM
chain = prompt | openai_llm

# Execute the chain with a specific product type
response = chain.invoke({"product": "smart home devices"})

# Display the generated company name
print("OpenAI response:", response)

In [7]:
# ============================================================================
# LangChain RAG Tool Setup - Cell 2: Retrieval-Augmented Generation (RAG)
# ============================================================================

# ============================================================================
# Step 1: Load and Process Document
# ============================================================================
# Read the sample.txt file containing text data to be indexed
with open("sample.txt", "r", encoding="utf-8") as f:
    text_data = f.read()

# ============================================================================
# Step 2: Split Text into Chunks
# ============================================================================
# Initialize text splitter to break large documents into manageable chunks
# - separator="\n": Split on newline characters
# - chunk_size=300: Each chunk contains ~300 characters
# - chunk_overlap=50: Chunks overlap by 50 characters to maintain context
splitter = CharacterTextSplitter(separator="\n", chunk_size=300, chunk_overlap=50)
docs = splitter.create_documents([text_data])

# ============================================================================
# Step 3: Create Embeddings and Vector Store
# ============================================================================
# Initialize OpenAI embeddings (converts text to numerical vectors)
embeddings = OpenAIEmbeddings()

# Create a FAISS vector database from the document chunks
# FAISS enables fast similarity search on the embedded documents
vectorstore = FAISS.from_documents(docs, embeddings)

# Create a retriever that fetches the top 4 most relevant documents
# when a query is performed (search_kwargs={"k": 4})
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# ============================================================================
# Step 4: Create RAG Tool
# ============================================================================
# Convert the retriever into a tool that an agent can use
# This tool will retrieve relevant documents from sample.txt
rag_tool = create_retriever_tool(
    retriever,
    name="langchain_rag",  # Tool name for the agent
    description="Use this to answer questions using the content in sample.txt"  # Tool description
)

# ============================================================================
# Step 5: Create an Agent with Tools
# ============================================================================
# Create a modern LangChain agent that can use tools (like our RAG tool)
# This replaces the older create_tool_calling_agent + AgentExecutor pattern
agent = create_agent(
    openai_llm,  # Use OpenAI's LLM to power the agent
    tools=[rag_tool],  # Provide the RAG tool for document retrieval
    system_prompt="You are a helpful assistant. Use tools when needed."  # Agent behavior instruction
)

# ============================================================================
# Step 6: Invoke the Agent
# ============================================================================
# Execute the agent with a user query using the messages format
# The agent will automatically decide when to use the RAG tool
result = agent.invoke({
    "messages": [{"role": "user", "content": "What is LangChain and why do we use it?"}]
})

# Extract and display the final answer from the agent's response
# The last message in the response contains the agent's final answer
print(result["messages"][-1].content)

LangChain is a framework for building applications with Large Language Models (LLMs). It was created by Harrison Chase and supports RAG, agents, memory, tools, and more. LangChain is commonly used in chatbots, document Q&A, and AI workflow.


In [ ]:
# ============================================================================
# WHY WE NEED CONTEXTUAL COMPRESSION RETRIEVER
# ============================================================================
# Understanding the problem it solves and when to use it

from tabulate import tabulate

print("\n" + "█"*150)
print("█" + " "*148 + "█")
print("█" + "CONTEXTUAL COMPRESSION RETRIEVER: SOLVING REAL-WORLD PROBLEMS".center(148) + "█")
print("█" + " "*148 + "█")
print("█"*150 + "\n")

# ============================================================================
# DIAGRAM 1: THE PROBLEM
# ============================================================================
print("┌" + "─"*148 + "┐")
print("│" + "DIAGRAM 1: THE PROBLEM - WASTED TOKENS & NOISE".ljust(148) + "│")
print("└" + "─"*148 + "┘\n")

problem_diagram = """
USER QUERY: "What is LangChain?"

┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│  REGULAR RETRIEVER (WITHOUT COMPRESSION)                                                    │
└─────────────────────────────────────────────────────────────────────────────────────────────┘

Retrieved Document 1 (500 tokens):
┌─────────────────────────────────────────────────────────────────────────────────────────┐
│ LangChain was founded in 2022... [100 words about funding]... Series A funding announced │
│ in 2023... [50 words about investors]... Headquarters in San Francisco... [lots more      │
│ irrelevant details]... ✓ KEY INFO: "LangChain is a framework for LLMs"                   │
│ [200 more words about company history]...                                                 │
└─────────────────────────────────────────────────────────────────────────────────────────┘

🔴 Problem:
  • 500 tokens sent to LLM
  • Only 1-2 sentences are relevant
  • 95% is NOISE/WASTED tokens
  • Costs: $0.15 per 1000 tokens (GPT-4) → Wastes money!
  • LLM context gets cluttered
  • Potential for hallucination from irrelevant content


┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│  CONTEXTUAL COMPRESSION RETRIEVER (WITH COMPRESSION)                                        │
└─────────────────────────────────────────────────────────────────────────────────────────────┘

Same Retrieved Document (500 tokens) → LLM Compression:

"Extract ONLY the parts relevant to: 'What is LangChain?'"

Result (50 tokens):
┌─────────────────────────────────────────────────────────────────────────────────────────┐
│ LangChain is a framework for building applications with Large Language Models. It was   │
│ founded in 2022 and supports RAG, agents, memory, tools, and more.                      │
└─────────────────────────────────────────────────────────────────────────────────────────┘

✅ Solution:
  • Only 50 tokens sent to LLM
  • 100% relevant content
  • 90% token reduction!
  • Saves money: 9x cheaper for this retrieval
  • Cleaner context = Better answers
  • Less hallucination risk
"""

print(problem_diagram)

# ============================================================================
# DIAGRAM 2: TOKEN COST COMPARISON
# ============================================================================
print("\n┌" + "─"*148 + "┐")
print("│" + "DIAGRAM 2: REAL-WORLD TOKEN & COST COMPARISON".ljust(148) + "│")
print("└" + "─"*148 + "┘\n")

cost_diagram = """
Scenario: 10 queries to a knowledge base with 100-doc context

┌──────────────────────────────────────────────────────────────────────────────────────────────┐
│ REGULAR RETRIEVER (No Compression)                                                           │
├──────────────────────────────────────────────────────────────────────────────────────────────┤
│ Per Query:                                                                                   │
│   • Retrieve 4 documents × 500 tokens average = 2,000 tokens                               │
│   • Send to LLM = 2,000 tokens (input)                                                      │
│   • LLM response = 200 tokens (output)                                                      │
│   • Total per query = 2,200 tokens                                                          │
│                                                                                              │
│ For 10 queries:                                                                              │
│   • Total tokens = 22,000 tokens                                                            │
│   • Cost (GPT-4): 22,000 × $0.03/1K = $0.66                                                │
│   • Cost (GPT-4 Turbo): 22,000 × $0.01/1K = $0.22                                          │
└──────────────────────────────────────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────────────────────────────────────┐
│ CONTEXTUAL COMPRESSION RETRIEVER (With Compression)                                          │
├──────────────────────────────────────────────────────────────────────────────────────────────┤
│ Per Query:                                                                                   │
│   • Retrieve 4 documents = 2,000 tokens (not sent to LLM, used for compression)             │
│   • LLM compresses → 250 tokens average (90% reduction)                                     │
│   • Send compressed docs to LLM = 250 tokens (input)                                        │
│   • LLM response = 200 tokens (output)                                                      │
│   • Compression cost = 2,000 + 250 = 2,250 tokens                                           │
│   • Response cost = 450 tokens                                                              │
│   • Total per query = 2,700 tokens (includes compression overhead)                          │
│                                                                                              │
│ For 10 queries:                                                                              │
│   • Total tokens = 27,000 tokens                                                            │
│   • BUT: Only 4,500 tokens sent to main LLM                                                │
│   • Main LLM cost (GPT-4): 4,500 × $0.03/1K = $0.135                                       │
│   • Compression LLM cost: 22,500 × $0.0005/1K (cheaper model) = $0.011                     │
│   • Total cost ≈ $0.15 (70% SAVINGS!)                                                      │
└──────────────────────────────────────────────────────────────────────────────────────────────┘

💰 SAVINGS: $0.66 → $0.15 (77% cheaper!)
"""

print(cost_diagram)

# ============================================================================
# DIAGRAM 3: COMPARISON TABLE
# ============================================================================
print("\n┌" + "─"*148 + "┐")
print("│" + "DIAGRAM 3: DETAILED COMPARISON".ljust(148) + "│")
print("└" + "─"*148 + "┘\n")

comparison_data = [
    ["Aspect", "Regular Retriever", "Contextual Compression"],
    ["─" * 30, "─" * 50, "─" * 60],
    [
        "Tokens per 4 docs",
        "~2,000 tokens",
        "~250 tokens (90% reduction)"
    ],
    [
        "LLM Context",
        "Cluttered with irrelevant info",
        "Clean, focused, relevant only"
    ],
    [
        "Cost per query",
        "High (full documents)",
        "Low (compressed documents)"
    ],
    [
        "Processing Speed",
        "Fast ⚡⚡⚡",
        "Slower ⚡ (needs compression)"
    ],
    [
        "Hallucination Risk",
        "Higher (noise confuses LLM)",
        "Lower (only relevant info)"
    ],
    [
        "Best For",
        "Simple queries, quick responses",
        "Large docs, token-limited, cost-sensitive"
    ],
    [
        "Setup Complexity",
        "Simple",
        "Moderate (needs compressor LLM)"
    ],
    [
        "Trade-off",
        "Fast but expensive",
        "Slower but cheaper & cleaner"
    ]
]

print(tabulate(comparison_data, tablefmt="grid", maxcolwidths=[30, 50, 60]))

# ============================================================================
# DIAGRAM 4: WHEN TO USE EACH
# ============================================================================
print("\n\n┌" + "─"*148 + "┐")
print("│" + "DIAGRAM 4: DECISION GUIDE - WHEN TO USE WHAT".ljust(148) + "│")
print("└" + "─"*148 + "┘\n")

decision_guide = """
┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│                           CHOOSE REGULAR RETRIEVER WHEN:                                     │
├─────────────────────────────────────────────────────────────────────────────────────────────┤
│  ✓ Small documents (< 200 tokens average)                                                   │
│  ✓ Need maximum speed (no compression overhead)                                             │
│  ✓ Budget is unlimited                                                                      │
│  ✓ Few documents to retrieve                                                                │
│  ✓ Simple Q&A with high relevance documents                                                 │
│  ✓ Prototyping / Learning (quick setup)                                                     │
└─────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│                    CHOOSE CONTEXTUAL COMPRESSION WHEN:                                       │
├─────────────────────────────────────────────────────────────────────────────────────────────┤
│  ✓ Large documents (> 500 tokens)                                                           │
│  ✓ Token limits (context window constraints)                                                │
│  ✓ Cost-sensitive (production, many queries)                                                │
│  ✓ Documents have mostly irrelevant information                                             │
│  ✓ Need high-quality, focused context                                                       │
│  ✓ Want to reduce hallucination                                                             │
│  ✓ Running on budget (startups, limited resources)                                          │
└─────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────┐
│                            HYBRID APPROACH:                                                   │
├─────────────────────────────────────────────────────────────────────────────────────────────┤
│  Use both: Retrieve 10 docs quickly → Compress top 4 → Send to LLM                         │
│  Best of both worlds: Good recall + low cost                                                │
└─────────────────────────────────────────────────────────────────────────────────────────────┘
"""

print(decision_guide)

# ============================================================================
# CODE EXAMPLE: HOW TO USE IT
# ============================================================================
print("\n┌" + "─"*148 + "┐")
print("│" + "DIAGRAM 5: CODE EXAMPLE - IMPLEMENTING CONTEXTUAL COMPRESSION".ljust(148) + "│")
print("└" + "─"*148 + "┘\n")

code_example = """
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMCompressor
from langchain_openai import ChatOpenAI

# Step 1: Create base retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# Step 2: Create compressor (LLM that compresses documents)
compressor_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
compressor = LLMCompressor.from_llm(compressor_llm)

# Step 3: Wrap retriever with compression
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# Step 4: Use it (same way as regular retriever)
docs = compression_retriever.invoke("What is LangChain?")

print(f"Retrieved {len(docs)} documents")
for doc in docs:
    print(f"Compressed content: {doc.page_content}")
    print(f"Original size was larger, now optimized!")


═══════════════════════════════════════════════════════════════════════════════════════════════

WHAT HAPPENS INTERNALLY:

1. Your Query
   ↓
2. Base Retriever fetches 10 raw documents (5,000 tokens)
   ↓
3. LLMCompressor analyzes: "Which parts are relevant to the query?"
   ↓
4. Compressor creates summaries/extracts relevant parts
   ↓
5. Compressed documents returned (500 tokens)
   ↓
6. Documents sent to your LLM with 10x fewer tokens!
"""

print(code_example)

print("\n" + "█"*150 + "\n")

In [ ]:
# ============================================================================
# Retriever Types & When to Use Them - Educational Guide
# ============================================================================
# This cell demonstrates different retriever strategies available in LangChain

# Print header
print("\n" + "="*80)
print("LANGCHAIN RETRIEVER TYPES & USE CASES")
print("="*80 + "\n")

# ============================================================================
# 1. SIMILARITY SEARCH (Default)
# ============================================================================
print("1. SIMILARITY SEARCH (Default Retriever)")
print("-" * 80)
print("What it does:")
print("  • Returns the k most similar documents based on vector similarity")
print("  • Ranks results by relevance score (highest to lowest)")
print("\nWhen to use:")
print("  ✓ General use cases and standard applications")
print("  ✓ When you want straightforward relevance ranking")
print("  ✓ Most RAG applications start with this")
print("\nPros:")
print("  • Fast and efficient")
print("  • Simple to understand and use")
print("  • Works well for most scenarios")
print("\nCons:")
print("  • May return redundant/similar results")
print("  • No built-in quality filtering")
print("\nExample:")
similarity_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)
print("  retriever = vectorstore.as_retriever(")
print("      search_type='similarity',")
print("      search_kwargs={'k': 4}")
print("  )")
print()

# ============================================================================
# 2. MAXIMUM MARGINAL RELEVANCE (MMR)
# ============================================================================
print("2. MAXIMUM MARGINAL RELEVANCE (MMR) RETRIEVER")
print("-" * 80)
print("What it does:")
print("  • Returns diverse results by reducing redundancy")
print("  • Balances relevance with diversity")
print("  • Avoids returning very similar documents")
print("\nWhen to use:")
print("  ✓ Want diverse perspectives on a topic")
print("  ✓ Building recommendation systems")
print("  ✓ Need comprehensive coverage, not just top matches")
print("\nPros:")
print("  • More diverse search results")
print("  • Better for exploring multiple angles")
print("  • Reduces information redundancy")
print("\nCons:")
print("  • Slightly slower than similarity search")
print("  • Requires tuning fetch_k parameter")
print("\nExample:")
mmr_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 4,        # Final number of documents to return
        "fetch_k": 20  # Consider top 20 for diversity
    }
)
print("  retriever = vectorstore.as_retriever(")
print("      search_type='mmr',")
print("      search_kwargs={")
print("          'k': 4,        # Return 4 documents")
print("          'fetch_k': 20  # Consider top 20 candidates")
print("      }")
print("  )")
print()

# ============================================================================
# 3. SIMILARITY WITH SCORE THRESHOLD
# ============================================================================
print("3. SIMILARITY WITH SCORE THRESHOLD")
print("-" * 80)
print("What it does:")
print("  • Returns only documents with similarity above a threshold")
print("  • Filters out low-quality matches automatically")
print("  • Returns 0 results if nothing meets the threshold")
print("\nWhen to use:")
print("  ✓ Need quality guarantees")
print("  ✓ Want to avoid low-relevance results")
print("  ✓ Building strict compliance/safety systems")
print("\nPros:")
print("  • Guaranteed minimum quality")
print("  • Prevents hallucination from poor matches")
print("  • Clear quality control")
print("\nCons:")
print("  • May return fewer results than expected")
print("  • Requires tuning score_threshold")
print("\nExample:")
threshold_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.5,  # Only return if similarity > 0.5
        "k": 4
    }
)
print("  retriever = vectorstore.as_retriever(")
print("      search_type='similarity_score_threshold',")
print("      search_kwargs={")
print("          'score_threshold': 0.5,  # Minimum similarity")
print("          'k': 4")
print("      }")
print("  )")
print()

# ============================================================================
# VECTOR STORE OPTIONS
# ============================================================================
print("VECTOR STORE OPTIONS IN LANGCHAIN")
print("-" * 80)
print("Different vector databases you can use instead of FAISS:\n")

vector_stores = {
    "FAISS": {
        "Speed": "⚡⚡⚡⚡⚡ Fastest",
        "Storage": "In-memory only",
        "Persistence": "No",
        "Use Case": "Development, prototyping",
        "Best For": "Quick experiments, local testing"
    },
    "Chroma": {
        "Speed": "⚡⚡⚡⚡ Fast",
        "Storage": "Persistent",
        "Persistence": "Yes",
        "Use Case": "Development + persistence",
        "Best For": "Notebooks, small projects with storage"
    },
    "Pinecone": {
        "Speed": "⚡⚡⚡⚡ Fast",
        "Storage": "Cloud-based",
        "Persistence": "Yes",
        "Use Case": "Production, large scale",
        "Best For": "Production systems, enterprise apps"
    },
    "Weaviate": {
        "Speed": "⚡⚡⚡⚡ Fast",
        "Storage": "Cloud + Self-hosted",
        "Persistence": "Yes",
        "Use Case": "Production, hybrid search",
        "Best For": "Semantic + keyword search"
    },
    "Qdrant": {
        "Speed": "⚡⚡⚡⚡⚡ Fastest",
        "Storage": "Cloud + Self-hosted",
        "Persistence": "Yes",
        "Use Case": "High-performance production",
        "Best For": "Large-scale, performance-critical apps"
    },
}

for store_name, features in vector_stores.items():
    print(f"\n{store_name}:")
    for key, value in features.items():
        print(f"  {key}: {value}")

print("\n")

# ============================================================================
# QUICK DECISION GUIDE
# ============================================================================
print("="*80)
print("QUICK DECISION GUIDE - WHICH RETRIEVER TO USE?")
print("="*80)
print("""
Your Situation                  | Recommended Retriever
──────────────────────────────────────────────────────────────────────────────
Building a prototype/demo       | Similarity Search + FAISS
──────────────────────────────────────────────────────────────────────────────
Need diverse search results     | MMR Retriever
──────────────────────────────────────────────────────────────────────────────
Must filter low-quality results | Similarity with Score Threshold
──────────────────────────────────────────────────────────────────────────────
Production system with storage  | Similarity Search + Chroma/Pinecone
──────────────────────────────────────────────────────────────────────────────
High-performance production     | Similarity Search + Qdrant
──────────────────────────────────────────────────────────────────────────────
Hybrid keyword + semantic       | Ensemble Retriever (BM25 + Vector)
──────────────────────────────────────────────────────────────────────────────
Complex questions, need recall  | Multi-Query Retriever
──────────────────────────────────────────────────────────────────────────────
""")

print("="*80)
print("Current Setup in this Notebook:")
print("="*80)
print("Vector Store: FAISS (in-memory, fast)")
print("Retriever Type: Similarity Search (default)")
print("Number of Results: k=4 (returns top 4 most similar documents)")
print("\n✓ Good for: Learning, prototyping, quick experiments")
print("✗ Not suitable for: Production systems needing persistence")
print("="*80 + "\n")